In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
indexing_tables = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tf.__version__

In [ ]:
raw_articles = pd.read_csv('dataset/test_articles_dataset_newyork.csv', delimiter=',')

In [ ]:
embedding_model = gs.models.FastText.load('../train_embedding_models/fasttext_embedding_50d_all_signals')
# embedding_model = gs.models.FastText.load('train_embedding_model/fasttext_embedding_50d_all_signals_newyorkdata')

In [ ]:
indexing_tables.indices.close(index='data_table_newyork')
indexing_tables.indices.put_settings(index='data_table_newyork', body={"index": {"similarity": {"default": {"type": "BM25"}}}})
indexing_tables.indices.open(index='data_table_newyork')

In [ ]:
def search_indexing(query):
    
    result= indexing_tables.search(
        index="data_table_newyork", 
        body = {
        "_source": ["table_url","table_page_title","table_page_content","table_page_keywords"],
        "from" : 0,
        "size" : 100,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["table_page_title","table_page_content","table_page_keywords"] 
            }
        }
    })
    
    return result

In [ ]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID == ID_goal:
    
            accuracy = 1
            break;
            
    return accuracy

In [ ]:
MAX_PAD = 55

def sequence_padding(X_DIM, value):
    
    value_padding = np.pad(value, ((0,MAX_PAD - X_DIM),(0,0)), 'constant')
    
    return value_padding

In [ ]:
def create_embedding(value):

    value = tknzr.tokenize(str(value))
    
    if len(value) < MAX_PAD:
        
        embedding = embedding_model.wv[value]
        embedding = embedding.astype('float16')
        
        padding_embedding = sequence_padding(embedding.shape[0],embedding)
        
        return padding_embedding
        
    else:
        
        embedding = embedding_model.wv[value[0:MAX_PAD]]
        embedding = embedding.astype('float16')
        
        return embedding

In [ ]:
def search_index(article_title):
    
    tables_index = []

    result_index = search_indexing(article_title)
        
    for hit in result_index['hits']['hits']:
    
        table_ID = hit['_source']['table_url']
        
        table_page_title = hit['_source']['table_page_title']
        
        table_page_cotent = hit['_source']['table_page_content'][0:1000]
        
        table_page_keywords = hit['_source']['table_page_keywords']
    
        tables_index.append([table_ID,table_page_title,table_page_cotent,table_page_keywords])
    
    return tables_index

In [ ]:
# affinity_model_title_main_passage_keywords_1_1_34_0.9841.h5 
# attention_model_title_main_passage_keywords_1_1_23_0.9844.h51
# coattention_model_title_main_passage_keywords_1_1_11_0.9621.h5

In [ ]:
ranking_model = tf.keras.models.load_model('../model_title_main_passage_keywords/coattention_model_title_main_passage_keywords_1_1_39_0.9740.h5')

In [ ]:
raw_articles.info()

In [ ]:
result = []

def run_search(k):
    
    TOP_K = k
    accuracy = []
    
    for i,row in tqdm(raw_articles.iterrows()):
    
        article_title = []
        article_main_passage = []
        article_keywords = []
        
        title_table = []
        table_page_content = []
        table_keywords = []
        
        ranked_tables_model = []
    
        article_ID = row['article_key_match']
        article_title_text = str(row['article_title'])
        article_main_passage_text = str(row['article_main_passage'][0:1000])
        article_meta_description_text = str(row['article_meta_description'])
        article_keywords_text = str(row['article_title'])
        
        catch = article_title_text+" "+article_main_passage_text+" "+article_keywords_text
        
        ranked_tables_index = search_index(catch)
        
        article_title_embedding = create_embedding(article_title_text)
        article_main_passage_embedding = create_embedding(article_main_passage_text)
        article_keywords_embedding = create_embedding(article_keywords_text)
        
        for table_ID, table_title_index, table_page_content_index, table_keywords_index in (ranked_tables_index):
            
            table_title_embedding = create_embedding(str(table_title_index))
            table_page_content_embedding = create_embedding(str(table_page_content_index))
            table_page_keywords_embedding = create_embedding(str(table_keywords_index))
            
            article_title.append(article_title_embedding)
            article_main_passage.append(article_main_passage_embedding)
            article_keywords.append(article_keywords_embedding)
            
            title_table.append(table_title_embedding)
            table_page_content.append(table_page_content_embedding)
            table_keywords.append(table_page_keywords_embedding)
    
        article_title = np.array(article_title)
        article_main_passage = np.array(article_main_passage)
        article_keywords = np.array(article_keywords)
        
        title_table = np.array(title_table)
        table_page_content = np.array(table_page_content)
        table_keywords = np.array(table_keywords)
    
        table_ranking_model = ranking_model.predict([article_title,article_main_passage,article_keywords,title_table,table_page_content,table_keywords])
    
        for i in range(0,len(table_ranking_model)):
        
            ranked_tables_model.append([ranked_tables_index[i][0],ranked_tables_index[i][1],table_ranking_model[i][0]]) 
        
        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking', ascending=False)
        final_ranked_tables = data_frame_sorting.iloc[0:TOP_K,0:1].values
        
#         selected_top = data_frame_sorting.head(TOP_K)
#         min_score = selected_top['table_ranking'].min()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] >= min_score]
#         final_ranked_tables = draw_tables_socres.iloc[:,0:1].values
           
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))
        
    result.append(["Acc@"+str(k),str(round(np.mean(accuracy),4))])

In [ ]:
accuracy_K = [1,5,10,20]

for k in accuracy_K:
     
    run_search(k)

In [ ]:
result